In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import neighbors
from os import listdir
from glob import glob
%matplotlib inline
"""

FOLDER STRUCTURE

dataset
    |
    |----------1_finger
    |        .   |-------hand_0.png
    |        .   |         .
    |        .   |         .
    |        .   |         .
    |        .   |-------hand_n.png
    |        .
    |----------5_fingers
    
"""

path = "dataset_28"
resize = True
percentage = 100
numb_label = []
im_size = 28

classes = listdir(path)

image_list = []
labels = []
for classe in classes:
    for filename in glob(path+'/'+classe+'/*.png'):
        
        image_list.append(cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2GRAY))
                
        label=np.zeros(len(classes))
        label[classes.index(classe)]=1
        labels.append(label)
        
indice = np.random.permutation(len(image_list))[:int(len(image_list)*percentage/100)]

images = np.array([image_list[x] for x in indice])
labels = np.array([labels[x] for x in indice])
classes = np.array(classes)

#creamos data, imagenes vectorizadas
nsamples, nx, ny = images.shape
data = images.reshape((nsamples,nx*ny))

#numb_label = etiquetas numericas
for i in range (0,len(image_list)):
    numb_label.append(labels[i].argmax())
numb_label=np.array(numb_label)

#ordenamos las etiquetas
for i in range (0,len(image_list)):
    if numb_label[i]==0: numb_label[i]=5
    elif numb_label[i]==1: numb_label[i]=1
    elif numb_label[i]==2: numb_label[i]=4
    elif numb_label[i]==3: numb_label[i]=3
    elif numb_label[i]==4: numb_label[i]=2
    

print "Working with {} images \n".format(len(image_list))

#Using the KNN Classifier
classifier = neighbors.KNeighborsClassifier()

X_train,y_train = data[:], numb_label[:]

#Fit model with training data
classifier.fit(X_train, y_train)

#vaiables para la caputura de video
cap_depth = cv2.VideoCapture(1)
x,y,w,h = 0,0,100,100

while True:

    ret, frame_depth = cap_depth.read()
    frame_depth = cv2.flip(frame_depth,1)
    lower_green = np.array([0, 154, 0])
    upper_green = np.array([0, 154, 0])
    mask = cv2.inRange(frame_depth, lower_green, upper_green)
    median = cv2.medianBlur(mask ,15)
    median = 255 - median;
    resized_image = cv2.resize(median, (200, 150))
    
    median, contours, hierarchy = cv2.findContours(median,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) != 0:
        # Dibujamos los contornos
        #cv2.drawContours(median, contours, -1, (122,122,0), 3)

        #Buscamos el area maxima
        c = max(contours, key = cv2.contourArea)
        
        #Declaramos las medidas del rectangulo de contorno maximo
        x,y,w,h = cv2.boundingRect(c)
        
        #Dibujamos el contorno
        cv2.rectangle(median,(x,y),(x+w,y+h),(255,255,255),2)
        
    roi_max = median[y:y+h,x:x+w]
    
    resized_roi = cv2.resize(roi_max, (im_size, im_size))
    cv2.imshow('median', median)

    cv2.imshow('resized_roi', resized_roi)
    
    
    nx, ny = resized_roi.shape
    data = resized_roi.reshape(nx*ny)
    data = resized_roi.reshape(1, -1)
    
    
    #printing example
    result = classifier.predict(data)
    
    cv2.putText(roi_max, str(result), (15,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,150,150),2)
    cv2.imshow('roi_max', roi_max)
    #salimos de la ejecucion con q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap_depth.release()
cv2.destroyAllWindows()
cv2.waitKey(0)

Working with 152 images 



-1